In [1]:
import json

# Wikidata

In [2]:
with open('wikidata/wizards/data/info.json') as f:
    info = json.load(f)

In [3]:
with open('wikidata/wizards/data/relation.json') as f:
    relation = json.load(f)

In [35]:
info[0]

{'character': 'http://www.wikidata.org/entity/Q173998',
 'characterLabel': 'Ron Weasley',
 'genderLabel': 'male',
 'blood_statusLabel': 'pureblood',
 'DOBLabel': '1980-03-01T00:00:00Z',
 'POBLabel': 'Devon',
 'hair_colorLabel': 'red hair',
 'educationLabel': 'Hogwarts',
 'employerLabel': "Weasleys' Wizard Wheezes",
 'membershipLabel': "Dumbledore's Army"}

## merge entity wikidata

- needeed merged first since there the way wikidata display one entity with different attributes values
- below is an example with entity `Ron Weasley` with `red hair` and `orange hair`

In [4]:
info[0]['characterLabel'],info[0]['hair_colorLabel'],info[1]['characterLabel'],info[1]['hair_colorLabel']

('Ron Weasley', 'red hair', 'Ron Weasley', 'orange hair')

### attritbutes

In [5]:
info[0].keys()

dict_keys(['character', 'characterLabel', 'genderLabel', 'blood_statusLabel', 'DOBLabel', 'POBLabel', 'hair_colorLabel', 'educationLabel', 'employerLabel', 'membershipLabel'])

In [27]:
wikidata = {}

indicator = -1
for i,entity0 in enumerate(info):
    if i < indicator:
        continue
        
    name = entity0['characterLabel']
    wikidata[name] = {}
    
    idx = [i]
    for j,entity1 in enumerate(info[i+1:]):
        if entity1['characterLabel'] == name:
            idx.append(j+i+1)
        else:
            indicator = j+i+1
            break
            
    wikidata[name]['info'] = idx

In [16]:
relation[0].keys()

dict_keys(['character', 'characterLabel', 'fatherLabel', 'motherLabel', 'spouseLabel', 'childLabel', 'siblingLabel', 'in_relationshipLabel', 'enemyLabel'])

In [29]:
indicator = -1
for i,entity0 in enumerate(relation):
    if i < indicator:
        continue
        
    name = entity0['characterLabel']
    
    idx = [i]
    for j,entity1 in enumerate(relation[i+1:]):
        if entity1['characterLabel'] == name:
            idx.append(j+i+1)
        else:
            indicator = j+i+1
            break
            
    wikidata[name]['relation'] = idx

In [30]:
wikidata

{'Ron Weasley': {'info': [0, 1, 2, 3, 4, 5],
  'relation': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23]},
 'Hermione Granger': {'info': [6, 7, 8, 9, 10, 11],
  'relation': [24, 25, 26, 27, 28, 29, 30, 31]},
 'Lord Voldemort': {'info': [12, 13, 14, 15, 16, 17],
  'relation': [32, 33, 34]},
 'Severus Snape': {'info': [18, 19, 20], 'relation': [35]},
 'Draco Malfoy': {'info': [21, 22, 23, 24], 'relation': [37]},
 'Ginny Weasley': {'info': [25, 26, 27],
  'relation': [38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73]},
 'Luna Lovegood': {'info': [28, 29], 'relation': [74]},
 'Neville Longbottom': {'info': [30, 31, 32], 'relation': [75]},
 'Minerva McGonagall': {'info': [33, 34], 'relat

# Fandom data

In [31]:
with open('fandom/fandom_data/Gryffindors.json') as f:
    Gryffindors = json.load(f)

In [32]:
Gryffindors[0].keys()

dict_keys(['name', 'Biographical information', 'Physical information', 'Relationship information', 'Magical characteristics', 'Affiliation'])

In [33]:
for k in list(Gryffindors[0].keys())[1:]:
    print(Gryffindors[0][k].keys())

dict_keys(['Born', 'Died', 'Blood status', 'Marital status', 'Nationality', 'Title(s)'])
dict_keys(['Species', 'Gender', 'Height', 'Weight', 'Hair colour', 'Eye colour'])
dict_keys(['Family members', 'Romances'])
dict_keys(['Boggart', 'Wand', 'Patronus'])
dict_keys(['Occupation', 'House', 'Loyalty'])


# Export Data

## entity attributes

| index | name  | blood_status |hair_color | eye_color | wand | patronus | occupation | loyalty | relation | house |
| :- | :-   | :-          | :-       | :-       | :-  | :-      | :-  | :-  | :- | -: |
| ***int*** | ***str*** |  ***str***| ***str***  |  ***str*** |  ***str***  |  ***str*** | ***list***  |    ***list***   |   ***list***  |  ***str*** | 






In [ ]:
results_ = {}